# Medical Image - TFM
<h4>subtitle: Generación de una tubería distribuida para la extracción de características en imágenes médicas patológicas</h4>
license: European Union Public Licence (EUPL) v1.2

<table>
  <tr> <td> author name: </td> <td> Israel Llorens </td> </tr>
  <tr> <td> email: </td> <td> sanchezis@hotmail.com </td> </tr>
</table>

<h7>date: 2024/03/22</h7>

---

In [1]:
%load_ext autoreload
%autoreload

In [2]:
import os
import sys

if './../lib' not in os.environ.get('PATH', '') or \
    os.path.join(  os.getcwd() , 'lib'  ) not in os.environ.get('PATH', ''):
    os.environ['PYTHONPATH'] = sys.executable
    os.environ['LIBPATH'] = os.pathsep + './../lib' + os.pathsep + '../lib' + os.pathsep + os.path.join( os.path.dirname(  os.getcwd() ), 'lib'  )
    os.environ['DYLD_LIBRARY_PATH'] = os.pathsep + './../lib' + os.pathsep + '../lib' + os.pathsep + os.path.join( os.path.dirname(  os.getcwd() ), 'lib'  )
    os.environ['LIBRARY_PATH'] = os.pathsep + './../lib' + os.pathsep + '../lib' + os.pathsep + os.path.join( os.path.dirname(  os.getcwd() ), 'lib'  )


In [3]:
os.environ['DYLD_LIBRARY_PATH']

':./../lib:../lib:/Users/illorens/Projects/source/Universidad/viu-MU-BD-and-DS-Image-Pathology/lib'

In [4]:
from openslide import OpenSlide

OSError: dlopen(./../lib/libopenslide.dylib, 0x0006): Library not loaded: /opt/homebrew/opt/libtiff/lib/libtiff.5.dylib
  Referenced from: <13BE9501-93DF-3B08-80E9-B0AD388B62B5> /Users/illorens/Projects/source/Universidad/viu-MU-BD-and-DS-Image-Pathology/lib/libopenslide.dylib
  Reason: tried: '/opt/homebrew/opt/libtiff/lib/libtiff.5.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libtiff/lib/libtiff.5.dylib' (no such file), '/opt/homebrew/opt/libtiff/lib/libtiff.5.dylib' (no such file), '/usr/lib/libtiff.5.dylib' (no such file, not in dyld cache), '/opt/homebrew/Cellar/libtiff/4.7.0/lib/libtiff.5.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/Cellar/libtiff/4.7.0/lib/libtiff.5.dylib' (no such file), '/opt/homebrew/Cellar/libtiff/4.7.0/lib/libtiff.5.dylib' (no such file), '/usr/lib/libtiff.5.dylib' (no such file, not in dyld cache)

In [2]:
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Initialize Spark with OpenSlide config
spark = SparkSession.builder \
    .appName("TIAToolboxWSI") \
    .config("spark.executorEnv.OPENSLIDE_PATH", "/usr/lib/openslide") \
    .config("spark.executorEnv.OBJC_DISABLE_INITIALIZE_FORK_SAFETY", "YES") \
    .getOrCreate()

@udf(BooleanType())
def verify_openslide():
    try:
        from openslide import OpenSlide
        return True
    except:
        return False

# Check installation across cluster
spark.range(10).withColumn("openslide_installed", verify_openslide()).show()    

25/03/23 23:55:53 WARN Utils: Your hostname, Airon.local resolves to a loopback address: 127.0.0.1; using 192.168.100.210 instead (on interface en0)
25/03/23 23:55:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/23 23:55:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---+-------------------+
| id|openslide_installed|
+---+-------------------+
|  0|              false|
|  1|              false|
|  2|              false|
|  3|              false|
|  4|              false|
|  5|              false|
|  6|              false|
|  7|              false|
|  8|              false|
|  9|              false|
+---+-------------------+

